# Работа с таблицами

### 1. Триггеры:
Вам доступна база данных, принадлежащая маркетплейсу Ozon. Она включает таблицы Customers, Orders и Notifications.  Первая таблица содержит информацию о покупателях магазина, вторая — информацию о заказанных товарах, третья — информацию об уведомлениях покупателей.

In [ ]:
+----+--------+
| id | name   |
+----+--------+
| 10 | Ivan   |
| 20 | John   |
| 30 | Mark   |
| 40 | Elon   |
| 50 | Pavel  |
| 60 | Andrey |
| 70 | Peter  |
| 80 | Ben    |
+----+--------+

In [ ]:
+----+-------------+------------+--------+-----------+
| id | customer_id | store      | amount | status    |
+----+-------------+------------+--------+-----------+
| 1  | 20          | Ozon       | 101    | Created   |
| 2  | 40          | PCUniverse | 799    | Shipped   |
| 3  | 10          | PCUniverse | 99     | Delivered |
| 4  | 30          | DarkStore  | 99     | Created   |
| 5  | 50          | Ozon       | 678    | Delivered |
| 6  | 10          | Ozon       | 458    | Created   |
+----+-------------+------------+--------+-----------+

In [ ]:
+----+----------+-------------------------------------------------------------+
| id | order_id | message                                                     |
+----+----------+-------------------------------------------------------------+
| 1  | 1        | John, new status of your delivery on 2024-08-26: Created    |
| 2  | 2        | Elon, new status of your delivery on 2024-08-26: Shipped    |
| 3  | 3        | Ivan, new status of your delivery on 2024-08-26: Delivered  |
| 4  | 4        | Mark, new status of your delivery on 2024-08-26: Created    |
| 5  | 5        | Pavel, new status of your delivery on 2024-08-26: Delivered |
| 6  | 6        | Ivan, new status of your delivery on 2024-08-26: Created    |
+----+----------+-------------------------------------------------------------+

Первое поле этой таблицы содержит идентификатор уведомления, второе — идентификатор заказа, третье — сообщение с уведомлением покупателя.

**Напишите запрос, создающий триггер, который после добавления нового заказа также добавляет сообщение для покупателя. При этом сообщение должно быть записано в следующем формате:**

<Имя покупателя>, new status of your delivery on <текущая дата>: <статус заказа>

                  
Также запрос должен включать создание триггера, который после обновления статуса заказа также обновляет сообщение для покупателя. При этом сообщение должно быть записано в том же формате, что и при добавлении заказа.

#### Решение:

``` sql
DELIMITER //
CREATE TRIGGER new_notification 
AFTER INSERT 
ON Orders 
FOR EACH ROW 
BEGIN
    INSERT INTO Notifications (order_id)
    SELECT id
    FROM Orders
    WHERE id = NEW.id;
    
    UPDATE Notifications
    SET message = CONCAT((SELECT name
                          FROM Customers
                          WHERE Customers.id = NEW.customer_id), ', new status of your delivery on ', CURDATE(), ': ', NEW.status)
     WHERE Notifications.order_id = NEW.id;
END //
            
CREATE TRIGGER update_notification 
AFTER UPDATE 
ON Orders 
FOR EACH ROW 
BEGIN
    UPDATE Notifications
    SET message = CONCAT((SELECT name
                          FROM Customers
                          WHERE Customers.id = NEW.customer_id), ', new status of your delivery on ', CURDATE(), ': ', NEW.status)
     WHERE Notifications.order_id = NEW.id;
END //
DELIMITER ;
```

### 2. Представления:
Вам доступна база данных, принадлежащая интернет-магазину Amazon. Она включает таблицу Orders, которая хранит информацию о заказах покупателей.

In [ ]:
+----+------------+----------+---------------+
| id | product_id | quantity | purchase_date |
+----+------------+----------+---------------+
| 1  | 1          | 7        | 2020-03-16    |
| 2  | 1          | 4        | 2020-05-02    |
| 3  | 1          | 7        | 2020-07-10    |
| 4  | 1          | 6        | 2021-02-23    |
| 5  | 1          | 5        | 2021-05-21    |
| 6  | 1          | 6        | 2021-10-11    |
| 7  | 3          | 5        | 2019-08-08    |
| 8  | 8          | 1        | 2021-06-03    |
| 9  | 2          | 5        | 2023-12-25    |
| 10 | 7          | 7        | 2022-07-10    |
| 11 | 3          | 5        | 2019-09-24    |
| 12 | 8          | 5        | 2022-07-29    |
| 13 | 7          | 5        | 2023-01-16    |
| 14 | 3          | 7        | 2022-12-21    |
| 15 | 3          | 3        | 2021-06-16    |
| 16 | 3          | 7        | 2019-06-11    |
| 17 | 8          | 3        | 2021-06-13    |
| 18 | 8          | 1        | 2020-05-25    |
| 19 | 3          | 3        | 2023-06-14    |
| 20 | 8          | 5        | 2022-01-13    |
| 21 | 7          | 9        | 2023-07-19    |
| 22 | 5          | 3        | 2023-12-01    |
| 23 | 4          | 8        | 2022-02-15    |
| 24 | 3          | 7        | 2020-09-06    |
| 25 | 8          | 3        | 2020-11-23    |
| 26 | 4          | 9        | 2023-10-05    |
| 27 | 7          | 2        | 2023-10-12    |
| 28 | 8          | 5        | 2020-01-03    |
| 29 | 6          | 6        | 2022-06-21    |
| 30 | 8          | 6        | 2021-07-01    |
| 31 | 3          | 4        | 2018-12-26    |
| 32 | 3          | 5        | 2019-08-11    |
| 33 | 3          | 7        | 2022-02-21    |
| 34 | 3          | 5        | 2018-09-04    |
| 35 | 7          | 3        | 2022-03-07    |
| 36 | 6          | 5        | 2021-01-01    |
| 37 | 3          | 4        | 2022-03-18    |
| 38 | 5          | 2        | 2023-01-31    |
| 39 | 8          | 7        | 2022-07-19    |
| 40 | 7          | 3        | 2022-10-27    |
| 41 | 2          | 6        | 2023-10-16    |
| 42 | 8          | 5        | 2020-08-04    |
| 43 | 4          | 7        | 2022-08-28    |
+----+------------+----------+---------------+

**Напишите запрос, создающий представление с именем PopularProducts. Представление должно определять уникальные идентификаторы товаров, которые были заказаны три или более раз в течение двух лет подряд.**

#### Решение:

``` sql
CREATE VIEW CountOrders AS
SELECT COUNT(id) AS orders, product_id, YEAR(purchase_date) AS year
FROM Orders 
GROUP BY product_id, YEAR(purchase_date);

CREATE VIEW PopularProducts AS
SELECT DISTINCT C1.product_id 
FROM CountOrders AS C1 JOIN CountOrders AS C2 ON C1.product_id = C2.product_id AND C1.year != C2.year
WHERE MOD(C1.year, C2.year) = 1 AND C1.orders >= 3 AND C2.orders >= 3;

```